In [1]:
import numpy as np

In [2]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


In [3]:
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-8
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [4]:
class FeedForwardNN:
    def __init__(self, input_size, hidden_size, learning_rate=0.1):
        self.lr = learning_rate
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, 1) * 0.01
        self.b2 = np.zeros((1, 1))

    def forward(self, X):
        self.Z1 = np.dot(X, self.W1) + self.b1
        self.A1 = relu(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2) + self.b2
        self.A2 = sigmoid(self.Z2)
        return self.A2

    def compute_loss(self, y_true, y_pred):
        return binary_cross_entropy(y_true, y_pred)

    def backward(self, X, y_true):
        m = X.shape[0]
        dZ2 = self.A2 - y_true
        dW2 = np.dot(self.A1.T, dZ2) / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2

    def train(self, X, y, epochs=1000, print_loss_every=100):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = self.compute_loss(y, y_pred)
            self.backward(X, y)

            if epoch % print_loss_every == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        y_pred = self.forward(X)
        return (y_pred > 0.5).astype(int)


In [5]:
if __name__ == "__main__":
    # OR Gate dataset
    X = np.array([[0,0], [0,1], [1,0], [1,1]])
    y = np.array([[0], [1], [1], [1]])

    # Create and train model
    model = FeedForwardNN(input_size=2, hidden_size=4, learning_rate=0.1)
    model.train(X, y, epochs=1000)

    # Predict
    preds = model.predict(X)
    print("Predictions:\n", preds)


Epoch 0, Loss: 0.6931
Epoch 100, Loss: 0.5614
Epoch 200, Loss: 0.4935
Epoch 300, Loss: 0.2838
Epoch 400, Loss: 0.1534
Epoch 500, Loss: 0.0924
Epoch 600, Loss: 0.0624
Epoch 700, Loss: 0.0459
Epoch 800, Loss: 0.0358
Epoch 900, Loss: 0.0291
Predictions:
 [[0]
 [1]
 [1]
 [1]]
